<a href="https://colab.research.google.com/github/vargopal/stockMarket/blob/F455--Quant-Finance/stockmarketML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#pip install --upgrade tensorflow
import tensorflow as tf
print(tf.__version__)

2.11.0


In [5]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [8]:
import pandas as pd
import numpy as np

# import dataset
fb_complete_data = pd.read_csv("/gdrive/My Drive/Colab Notebooks/META_Training.csv")

In [17]:
fb_complete_data.head()

#filtering open column
fb_training_processed = fb_complete_data[['Open']].values

In [22]:
# scaling features
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range = (0,1))
# still not entirely sure what the MinMaxScaler does but it essentially is placing the value as a decimal on a range between 0 and 1 representing where it is in comparison to other values
fb_training_scaled = scaler.fit_transform(fb_training_processed)

len(fb_training_scaled)

251

In [27]:
fb_training_features = []
fb_training_labels = []

# I tweaked the training features containing data of the last 30 days as opposed to 60 in the book
# Training labels should contain data of the 31st day

# This for loop essentially takes a value i and essentially gets the 30 values prior to it so it can predict that i value in the model
for i in range(30,len(fb_training_scaled)):
  fb_training_features.append(fb_training_scaled[i-30:i,0])
  fb_training_labels.append(fb_training_scaled[i,0])


In [38]:
# Converting data into numpy arrays
X_train = np.array(fb_training_features)
y_train = np.array(fb_training_labels)

print(X_train.shape, y_train.shape)

# Converting data into 3D shape
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1],1))


(221, 30) (221,)


In [40]:
# importing packages, need to learn what some of them mean
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Activation, Dense, Flatten, Dropout, Flatten, LSTM
from tensorflow.keras.models import Model

In [41]:
# defining the LSTM network

input_layer = Input(shape = (X_train.shape[1],1))
lstm1 = LSTM(100, activation = 'relu', return_sequences=True)(input_layer)